In [10]:
def stitch_images(image1, image2, xform, translation):
  """Stitch two matched images given the transformation between them.

  Args:
  - image1 (3D uint8 array): A color image.
  - image2 (3D uint8 array): A color image.
  - xform (2D float64 array): A 3x3 matrix representing the transformation
      between image1 and image2. This transformation should map coordinates
      in image1 to the corresponding coordinates in image2.

  Returns:
  - image_stitched (3D uint8 array)
  """
  height1, width1, _ = image1.shape
  height2, width2, _ = image2.shape

  # Transformed 4 corners of image1
  x1, y1, z1 = tuple(xform @ np.array([       0,         0, 1]).reshape(-1, 1))# + translation
  x2, y2, z2 = tuple(xform @ np.array([width1-1,         0, 1]).reshape(-1, 1))# + translation
  x3, y3, z3 = tuple(xform @ np.array([       0, height1-1, 1]).reshape(-1, 1))# + translation
  x4, y4, z4 = tuple(xform @ np.array([width1-1, height1-1, 1]).reshape(-1, 1))# + translation

  # Find appropriate size of stitched image
  min_x = int(min(x1/z1, x2/z2, x3/z3, x4/z4, 0))
  max_x = int(max(x1/z1, x2/z2, x3/z3, x4/z4, width2-1))
  min_y = int(min(y1/z1, y2/z2, y3/z3, y4/z4, 0))
  max_y = int(max(y1/z1, y2/z2, y3/z3, y4/z4, height2-1))

  new_width = max_x - min_x + 1
  new_height = max_y - min_y + 1

  # Addtional translation for shifting images to fit in final stitched image
#   translate = (np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]]) + translation).astype(np.float32)
  translate = (np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]])).astype(np.float32)

  image1_transformed = cv2.warpPerspective(image1, translate @ xform, (new_width, new_height))
  image2_transformed = cv2.warpPerspective(image2, translate, (new_width, new_height))

  image_stitched = cv2.addWeighted(image1_transformed, 0.5, image2_transformed, 0.5, 0)

  return image_stitched



In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [19]:
img1 = cv2.imread("./export/out_1/d/frame_0100_d_1583869334951745986.png", cv2.IMREAD_COLOR)
img2 = cv2.imread("./export/out_1/rgb/frame_0100_rgb_1583869334953597453.png", cv2.IMREAD_COLOR)
img2 = cv2.applyColorMap(cv2.convertScaleAbs(img2, alpha=0.03), cv2.COLORMAP_JET)

In [20]:
r = np.array([
    [0.999994, 0.00334682, 0.000991736],
    [-0.00334019, 0.999973, -0.00660826],
    [-0.00101383, 0.0066049, 0.999978]
    ])
t = np.array([
    [14.9153],
    [-0.320785],
    [-0.523442] 
    ])

In [21]:
image_stitched = stitch_images(img2, img1, r, t)

In [22]:
# cv2.imshow('b',image_stitched)
# cv2.show()
cv2.imwrite('./meow.png', image_stitched)

True

In [23]:
image_stitched.shape

(486, 1815, 3)

In [17]:
img1.shape

(480, 640, 3)

In [18]:
img2.shape

(480, 640, 3)